In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [ ]:
data = pd.read_csv('eco_rating.csv')
data.head()

,user_id,item_id,ratings
0,1,28,2
1,1,86,4
2,1,94,4
3,1,146,3
4,2,17,2


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 849 entries, 0 to 848
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   user_id  849 non-null    int64
 1   item_id  849 non-null    int64
 2   ratings  849 non-null    int64
dtypes: int64(3)
memory usage: 20.0 KB


In [ ]:
# Mengetahui total unique dari kolom user_id & place_id
len(data.user_id.unique())
len(data.item_id.unique())

182

In [ ]:
# Mengetahui keberadaan data yang hilang pada dataframe

data.isnull().sum()

user_id    0
item_id    0
ratings    0
dtype: int64

In [ ]:
# Menghapus data duplikat di kolom place_id

data_prep = data.drop_duplicates('item_id')

In [ ]:
data_prep.describe()

,user_id,item_id,ratings
count,182.000000,182.000000,182.000000
mean,33.912088,92.500000,3.521978
std,29.649201,52.683014,0.896316
min,1.000000,2.000000,2.000000
25%,10.250000,47.250000,3.000000
50%,25.000000,92.500000,4.000000
75%,52.500000,137.750000,4.000000
max,154.000000,183.000000,5.000000


In [ ]:
# Membuat list dari nilai unique pada user_id & item_id

user_ids = data.user_id.unique().tolist()
place_ids = data.item_id.unique().tolist()

In [ ]:
# Konversi data user_id & item_id ke label encoder

user_to_user_encoded = {x:i for i, x in enumerate(user_ids)}
user_encoded_to_user = {i: x for i, x in enumerate(user_ids)}

place_to_place_encoded = {x: i for i, x in enumerate(place_ids)}
place_encoded_to_place = {x: i for x, i in enumerate(place_ids)}

In [ ]:
data['user'] = data.user_id.map(user_to_user_encoded)
data['place'] = data.item_id.map(place_to_place_encoded)

data.head()

,user_id,item_id,ratings,user,place
0,1,28,2,0,0
1,1,86,4,0,1
2,1,94,4,0,2
3,1,146,3,0,3
4,2,17,2,1,4


In [ ]:
data['user_rating'] = data['ratings'].values.astype(np.float32)
data

,user_id,item_id,ratings,user,place,user_rating
0,1,28,2,0,0,2.0
1,1,86,4,0,1,4.0
2,1,94,4,0,2,4.0
3,1,146,3,0,3,3.0
4,2,17,2,1,4,2.0
...,...,...,...,...,...,...
844,156,144,4,155,88,4.0
845,156,15,5,155,46,5.0
846,156,29,4,155,77,4.0
847,156,52,4,155,155,4.0


In [ ]:
df = data.sample(frac=1,random_state=42)
df.head()

,user_id,item_id,ratings,user,place,user_rating
512,93,39,4,92,129,4.0
357,66,79,3,65,151,3.0
110,22,10,4,21,86,4.0
684,125,76,3,124,136,3.0
39,7,104,4,6,23,4.0


In [ ]:
x = df[['user','place']].values

y = df['user_rating'].apply(lambda x:(x-min(df.user_rating))/(max(df.user_rating-min(df.user_rating)))).values

train_indices = int(0.8 * df.shape[0])

x_train,x_val,y_train,y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:]
)

print(x,y)

[[ 92 129]
 [ 65 151]
 [ 21  86]
 ...
 [ 49  45]
 [ 78 108]
 [ 19  80]] [0.66666667 0.33333333 0.66666667 0.33333333 0.66666667 0.33333333
 0.66666667 0.33333333 0.         0.66666667 0.33333333 0.33333333
 0.         0.66666667 0.33333333 0.66666667 0.         0.33333333
 0.33333333 0.66666667 1.         0.33333333 0.66666667 0.33333333
 0.33333333 0.66666667 0.33333333 1.         0.         0.33333333
 0.         0.33333333 0.33333333 0.33333333 1.         0.66666667
 0.66666667 0.33333333 0.33333333 0.         0.66666667 0.66666667
 0.33333333 0.33333333 0.66666667 0.         0.         0.33333333
 0.33333333 0.         0.66666667 0.         0.66666667 0.66666667
 0.66666667 0.33333333 0.66666667 0.33333333 0.66666667 0.66666667
 0.33333333 0.66666667 0.33333333 0.66666667 0.66666667 0.33333333
 1.         1.         0.33333333 1.         0.         0.66666667
 0.66666667 0.         0.66666667 0.         0.66666667 0.33333333
 0.66666667 0.33333333 1.         0.66666667 0.33333333 0

In [ ]:
class RecommenderNet(tf.keras.Model):

  # Insialisasi fungsi
  def __init__(self, num_users, num_place, embedding_size, **kwargs):
    super(RecommenderNet, self).__init__(**kwargs)
    self.num_users = num_users
    self.num_place = num_place
    self.embedding_size = embedding_size
    self.user_embedding = tf.keras.layers.Embedding(
        num_users,
        embedding_size,
        embeddings_initializer = 'he_normal',
        embeddings_regularizer = tf.keras.regularizers.l2(1e-6)
    )
    self.user_bias = tf.keras.layers.Embedding(num_users, 1)
    self.place_embedding = tf.keras.layers.Embedding(
        num_place,
        embedding_size,
        embeddings_initializer = 'he_normal',
        embeddings_regularizer = tf.keras.regularizers.l2(1e-6)
    )
    self.place_bias = tf.keras.layers.Embedding(num_place, 1)

  def call(self, inputs):
    user_vector = self.user_embedding(inputs[:,0]) # memanggil layer embedding 1
    user_bias = self.user_bias(inputs[:, 0]) # memanggil layer embedding 2
    place_vector = self.place_embedding(inputs[:, 1]) # memanggil layer embedding 3
    place_bias = self.place_bias(inputs[:, 1]) # memanggil layer embedding 4

    dot_user_place = tf.tensordot(user_vector, place_vector, 2)

    x = dot_user_place + user_bias + place_bias

    return tf.nn.sigmoid(x) # activation sigmoid

In [ ]:
model = RecommenderNet(len(user_to_user_encoded), len(place_to_place_encoded), 100)

# model compile
model.compile(
    loss = 'mean_squared_error',
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=[tf.keras.metrics.MeanAbsoluteError(), tf.keras.metrics.RootMeanSquaredError()]
)

In [ ]:
history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 16,
    epochs = 100,
    validation_data = (x_val, y_val),
)

Epoch 1/100
43/43 [==============================] - 1s 9ms/step - loss: 0.1121 - mean_absolute_error: 0.2801 - root_mean_squared_error: 0.3342 - val_loss: 0.1174 - val_mean_absolute_error: 0.2805 - val_root_mean_squared_error: 0.3421
Epoch 2/100
43/43 [==============================] - 0s 4ms/step - loss: 0.1168 - mean_absolute_error: 0.2822 - root_mean_squared_error: 0.3411 - val_loss: 0.1175 - val_mean_absolute_error: 0.2810 - val_root_mean_squared_error: 0.3422
Epoch 3/100
43/43 [==============================] - 0s 3ms/step - loss: 0.1065 - mean_absolute_error: 0.2666 - root_mean_squared_error: 0.3257 - val_loss: 0.1185 - val_mean_absolute_error: 0.2834 - val_root_mean_squared_error: 0.3437
Epoch 4/100
43/43 [==============================] - 0s 3ms/step - loss: 0.1044 - mean_absolute_error: 0.2681 - root_mean_squared_error: 0.3225 - val_loss: 0.1198 - val_mean_absolute_error: 0.2852 - val_root_mean_squared_error: 0.3455
Epoch 5/100
43/43 [==============================] - 0s 3ms/

In [ ]:
result_metrics = model.evaluate(x_val, y_val)
print(f'MAE: {result_metrics[1]}\n RMSE: {result_metrics[2]}')

6/6 [==============================] - 0s 4ms/step - loss: 0.1382 - mean_absolute_error: 0.3063 - root_mean_squared_error: 0.3714
MAE: 0.30630621314048767
 RMSE: 0.37137171626091003
